In [12]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [14]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df= spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [15]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
total_votes_df = df.filter(df.total_votes >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [16]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [17]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51288086|R3LLICBKRHLRJC|B00USEZ7W4|     202667718|Seiki SE32HYT  LE...|Home Entertainment|          4|           17|         22|   Y|                N|Great picture, a ...|Very nice, sharp ...| 2015-08-27|
|         US|   50853411| RUD135S06DEVE|B00ZIGRX4W|     471919351|Samsung JS7000, S...|Home Entertainment|      

In [18]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [19]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
# total number of reviews
total_reviews = df.count()
total_reviews

373988

In [22]:
# number of 5-star reviews
star5_reviews = df.filter("star_rating = 5").count()
star5_reviews

373988

In [23]:
# percentage of 5-star reviews for the two types of review (paid vs unpaid)
total_paid_reviews = paid_df.count()
total_paid_reviews

261

In [24]:
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

106

In [25]:
paid_five_star_percent= round((paid_five_star_reviews / total_paid_reviews),2) * 100 
paid_five_star_percent

41.0

In [26]:
# percentage of 5-star reviews for the two types of review (paid vs unpaid)
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

24040

In [27]:
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

10899

In [28]:
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100 
unpaid_five_star_percent

45.3369384359401